In [1]:
import os
os.chdir('D:/Datasets/Brainwaves/ML/data')

In [2]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
os.chdir('D:/Datasets/Brainwaves/ML/public_test_data')
import xml.etree.ElementTree as ET
import re

path = 'D:/Datasets/Brainwaves/ML/public_test_data'

calculation_agent = []

for filename in os.listdir(path):
    if filename.endswith('.xml'): 
        tree = ET.parse(filename)
        root = tree.getroot()
        file=root.tag.split("}")[0]+"}"
        for x in root.iter(file+'formatting'):
            if( ("calculation agent is party" in x.text.lower()) | ("calculation agent. party a." in\
               x.text.lower()) ):
                ca=filename.split('.xml')[0]+"."+"Party "+(x.text.lower().split("party")[1][1].upper())
                calculation_agent.append(ca)  

In [4]:
df_ca=pd.DataFrame(calculation_agent).rename(columns={0:'calculation_agent'})
df_ca=pd.DataFrame(df_ca.calculation_agent.str.split('.',1).tolist(),
                                   columns = ['File_Name','calulation_agent'])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if '"threshold amount"' in elem.text.lower().replace(",",""):
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [8]:
df=pd.read_csv('../train_labels.csv')

In [9]:
df=df[['File_Name','Cross_default_threshold_type','Cross_default_percentage/amount','Cross_default_measure']]

In [10]:
train_thresh=pd.merge(features,df,how='left',on='File_Name')

In [11]:
train_thresh

,File_Name,Text,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure
0,100_bk,"(2) “""Threshold Amount"" means, in relation to ...",Percentage Formula,5,Shareholder's Equity
1,101_bk,"(2) “""Threshold Amount"" means, in relation to ...",Fixed Amount,25000000,NotFound
2,103_bk,"(2) “""Threshold Amount"" means, in relation to ...",Percentage Formula,5,Shareholder's Equity
3,104_bk,"(2) “""Threshold Amount"" means, in relation to ...",Percentage Formula,6,Shareholder's Equity
4,106_bk,"(2) “""Threshold Amount"" means, in relation to ...",The lesser of a fixed amount or a Percentage f...,10000000,NotFound
5,108_bk,"""Threshold Amount"" means 3% of the shareholder...",Percentage Formula,3,Shareholder's Equity
6,110_bk,"'Threshold Amount"" means USD 10,000,000 for Pa...",Fixed Amount,10000000,NotFound
7,113_bk,"'Threshold Amount"" means USD 10,000,000 for Pa...",Fixed Amount,10000000,NotFound
8,114_bk,"'Threshold Amount"" means USD 10,000,000 for Pa...",Percentage Formula,3,Shareholder's Equity
9,116_bk,"""Threshold Amount"" means USD 10,000,000 for Pa...",Fixed Amount,1000000,NotFound


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=True,sublinear_tf=True, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features_train = tfidf.fit_transform(train_thresh.Text)
labels1 = train_thresh.Cross_default_threshold_type
labels2 = train_thresh["Cross_default_percentage/amount"]
labels3 = train_thresh.Cross_default_measure

In [13]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if '"threshold amount"' in elem.text.lower().replace(",",""):
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [14]:
features_test = tfidf.transform(f_test['Text'])
import xgboost as xgb

xgb=xgb.XGBClassifier()
xgb.fit(features_train,labels1)
preds1 = xgb.predict(features_test) 
xgb.fit(features_train,labels2)
preds2 = xgb.predict(features_test)
xgb.fit(features_train,labels3)
preds3 = xgb.predict(features_test)

In [15]:
f_test['Cross_default_threshold_type']=preds1
f_test["Cross_default_percentage/amount"]=preds2
f_test['Cross_default_measure']=preds3

In [16]:
f_test

,File_Name,Text,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure
0,105_bk,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
1,109_bk,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
2,10_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
3,11_sh,"e) ""Threshold Amount"" means 10,000,000 U.S. Do...",Fixed Amount,10000000,NotFound
4,12_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
5,13_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
6,14_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
7,15_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound
8,16_bk,"""Threshold Amount"" means AS2,000,000 or its eq...",Fixed Amount,10000000,NotFound
9,16_sh,"""Threshold Amount"" means USD 10,000,000 or the...",Fixed Amount,10000000,NotFound


In [17]:
df_test = pd.merge(f_test,df_ca,how='left',on='File_Name')
df_test.drop('Text',axis=1,inplace=True)

In [18]:
df_test['calulation_agent'].fillna('Other',inplace=True)

In [19]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'the rating issued by standard' in elem.text.lower().replace(",",""):
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [20]:
df=pd.read_csv('../train_labels.csv')
train_thresh.drop('Text',axis=1,inplace=True)
train_thresh=pd.merge(features,df[['File_Name','RD_method_party_A']],how='left',on='File_Name')
features_train = tfidf.fit_transform(train_thresh.Text)
labels = train_thresh.RD_method_party_A

In [21]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'the rating issued by standard' in elem.text.lower().replace(",",""):
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [22]:
f_test['Text'].nunique()

12

In [23]:
features['Text'][0]

'"Threshold Amount" means USD 10,000,000 or the equivalent in any other currency for each ofParty A and Party B.'

In [24]:
features_test = tfidf.transform(f_test['Text'])
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()
lvc.fit(features_train,labels)
preds1 = lvc.predict(features_test)

In [25]:
f_test['RD_method_party_A']=preds1

In [26]:
df_test = pd.merge(df_test,f_test[['File_Name','RD_method_party_A']],on='File_Name',how='left')

In [27]:
df_test

,File_Name,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure,calulation_agent,RD_method_party_A
0,105_bk,Fixed Amount,10000000,NotFound,Party A,1
1,109_bk,Fixed Amount,10000000,NotFound,Party A,1
2,10_sh,Fixed Amount,10000000,NotFound,Party A,1
3,11_sh,Fixed Amount,10000000,NotFound,Party A,1
4,12_sh,Fixed Amount,10000000,NotFound,Party A,1
5,13_sh,Fixed Amount,10000000,NotFound,Party A,1
6,14_sh,Fixed Amount,10000000,NotFound,Party A,NotFound
7,15_sh,Fixed Amount,10000000,NotFound,Party A,NotFound
8,16_bk,Fixed Amount,10000000,NotFound,Party A,NotFound
9,16_sh,Fixed Amount,10000000,NotFound,Party A,NotFound


In [28]:
df_test.shape

(143, 6)

In [29]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'debt ' in elem.text.lower().replace(",",""):
                if 'long term' not in elem.text.lower().replace(",","").replace("-",""):
                    target_child=child
                    c=1
                    break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [30]:
df=pd.read_csv('../train_labels.csv')
train_thresh.drop('Text',axis=1,inplace=True)
train_thresh=pd.merge(features,df[['File_Name','RD_party_A_short_term_debt_classification']],how='left',on='File_Name')
features_train = tfidf.fit_transform(train_thresh.Text)
labels = train_thresh.RD_party_A_short_term_debt_classification

In [31]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'debt ' in elem.text.lower().replace(",",""):
                if 'long term' not in elem.text.lower().replace(",","").replace("-",""):
                    target_child=child
                    c=1
                    break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [32]:
features_test = tfidf.transform(f_test['Text'])
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()
lvc.fit(features_train,labels)
preds1 = lvc.predict(features_test)

In [33]:
f_test['RD_party_A_short_term_debt_classification']=preds1
df_test = pd.merge(df_test,f_test[['File_Name','RD_party_A_short_term_debt_classification']],on='File_Name',how='left')

In [34]:
df_test['RD_party_A_short_term_debt_classification'].fillna('NotFound',inplace=True)

In [35]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'debt ' in elem.text.lower().replace(",",""):
                if 'long term' in elem.text.lower().replace(",","").replace("-",""):
                    target_child=child
                    c=1
                    break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [36]:
df=pd.read_csv('../train_labels.csv')
train_thresh.drop('Text',axis=1,inplace=True)
train_thresh=pd.merge(features,df[['File_Name','RD_party_A_long_term_debt_classification']],how='left',on='File_Name')
features_train = tfidf.fit_transform(train_thresh.Text)
labels = train_thresh.RD_party_A_long_term_debt_classification

In [37]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'debt ' in elem.text.lower().replace(",",""):
                if 'long term' in elem.text.lower().replace(",","").replace("-",""):
                    target_child=child
                    c=1
                    break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [38]:
features_test = tfidf.transform(f_test['Text'])
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()
lvc.fit(features_train,labels)
preds1 = lvc.predict(features_test)

In [39]:
f_test['RD_party_A_long_term_debt_classification']=preds1
df_test = pd.merge(df_test,f_test[['File_Name','RD_party_A_long_term_debt_classification']],on='File_Name',how='left')

In [40]:
df_test['RD_party_A_long_term_debt_classification'].fillna('NotFound',inplace=True)

In [41]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'party a' in elem.text.lower().replace(",","").replace("-","").replace("(","").replace(")",""):
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [42]:
df=pd.read_csv('../train_labels.csv')
train_thresh.drop('Text',axis=1,inplace=True)
train_thresh=pd.merge(features,df[['File_Name','party_A']],how='left',on='File_Name')
features_train = tfidf.fit_transform(train_thresh.Text)
labels = train_thresh.party_A

In [43]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'party a' in elem.text.lower().replace(",","").replace("-","").replace("(","").replace(")",""):
                target_child=child
                c=1          
                break
            if c==1:
                break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [44]:
features_test = tfidf.transform(f_test['Text'])
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()
lvc.fit(features_train,labels)
preds1 = lvc.predict(features_test)

In [45]:
f_test['party_A']=preds1
df_test = pd.merge(df_test,f_test[['File_Name','party_A']],on='File_Name',how='left')

In [46]:
filenames=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/data") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'party b' in elem.text.lower().replace(",","").replace("-","").replace("(","").replace(")",""):
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

In [47]:
df=pd.read_csv('../train_labels.csv')
train_thresh.drop('Text',axis=1,inplace=True)
train_thresh=pd.merge(features,df[['File_Name','party B']],how='left',on='File_Name')
features_train = tfidf.fit_transform(train_thresh.Text)
labels = train_thresh['party B']

In [48]:
filenames_test=[name for name in os.listdir("D:/Datasets/Brainwaves/ML/public_test_data") if ".xml" in name]
f_test=pd.DataFrame()
c=0
f_test['File_Name']=["NotFound"]*len(filenames_test)
f_test['Text']=0
for i,filename in enumerate(filenames_test):
#     print(filename,":")
    tree=ET.parse("D:/Datasets/Brainwaves/ML/public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        f_test.loc[i,'File_Name']="19_bk"
    else:
        f_test.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if 'party b' in elem.text.lower().replace(",","").replace("-","").replace("(","").replace(")",""):
                target_child=child
                c=1
                break
            if c==1:
                break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    f_test.loc[i,'Text']=data

In [49]:
features_test = tfidf.transform(f_test['Text'])
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()
lvc.fit(features_train,labels)
preds1 = lvc.predict(features_test)

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

lvc=LogisticRegression()

X_train,X_test,y_train,y_test=train_test_split(features_train,labels,test_size=0.2)
lvc.fit(X_train,y_train)
preds=lvc.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(preds,y_test)

0.14

In [51]:
f_test['party B']=preds1
df_test = pd.merge(df_test,f_test[['File_Name','party B']],on='File_Name',how='left')

In [52]:
df_test.head()

,File_Name,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure,calulation_agent,RD_method_party_A,RD_party_A_short_term_debt_classification,RD_party_A_long_term_debt_classification,party_A,party B
0,105_bk,Fixed Amount,10000000,NotFound,Party A,1,NotFound,NotFound,SOCIETE GENERALE,Societe Generale
1,109_bk,Fixed Amount,10000000,NotFound,Party A,1,NotFound,NotFound,SOCIETE GENERALE,Company ABC
2,10_sh,Fixed Amount,10000000,NotFound,Party A,1,NotFound,NotFound,SOCIETE GENERALE,Societe Generale
3,11_sh,Fixed Amount,10000000,NotFound,Party A,1,NotFound,NotFound,SOCIETE GENERALE,Societe Generale
4,12_sh,Fixed Amount,10000000,NotFound,Party A,1,unsecured;unsubordinated,NotFound,SOCIETE GENERALE,Societe Generale


In [53]:
df_test=df_test.drop_duplicates(subset=['File_Name'])

In [54]:
df_cur_gov = pd.read_csv('../currency_governinglaw.csv')
df_cur_gov.drop('governing law',axis=1,inplace=True)
df_sai = pd.read_csv('../sai_final(3).csv')
df_sv = pd.read_csv('../saksham.csv')

In [55]:
df_sv.shape

(140, 10)

In [56]:
ss=pd.read_csv('../sample_submission.csv')

In [57]:
df_final_test2 = pd.merge(df_test,df_cur_gov,how='left',on='File_Name')
df_final_test2 = pd.merge(df_final_test2,df_sai,how='left',on='File_Name')
df_final_test2 = pd.merge(df_final_test2,df_sv,how='left',on='File_Name')

In [58]:
df_final_test2=pd.merge(ss[['File_Name']],df_final_test2,how='left',on='File_Name')

In [59]:
df_final_test2 = df_final_test2[['File_Name', 'date', 'party_A', 'party B', 'TC_currency', 'TC_Bespoke',
       'calulation_agent', 'CA_fallback_default_dispute',
       'CA_dispute_resolution ', 'RD_method_party_A',
       'RD_party_A_short_term_debt_classification',
       'RD_party_A_moody_short_term_trigger_method',
       'RD_party_A_moody_short_term_threshold',
       'RD_party_A_SnP_short_term_trigger_method',
       'RD_party_A_SnP_short_term_threshold',
       'RD_party_A_Fitch_short_term_trigger_method',
       'RD_party_A_Fitch_short_term_threshold',
       'RD_party_A_long_term_debt_classification',
       'RD_party_A_moody_long_term_trigger_method',
       'RD_party_A_moody_long_term_threshold',
       'RD_party_A_SnP_long_term_trigger_method',
       'RD_party_A_SnP_long_term_threshold',
       'RD_party_A_Fitch_long_term_trigger_method',
       'RD_party_A_Fitch_long_term_threshold', 'Cross_default_threshold_type',
       'Cross_default_percentage/amount', 'Cross_default_measure',
       'multibranch_party_A', 'multibranch_party_B', 'governing law']]

In [61]:
df_final_test2

,File_Name,date,party_A,party B,TC_currency,TC_Bespoke,calulation_agent,CA_fallback_default_dispute,CA_dispute_resolution,RD_method_party_A,...,RD_party_A_SnP_long_term_trigger_method,RD_party_A_SnP_long_term_threshold,RD_party_A_Fitch_long_term_trigger_method,RD_party_A_Fitch_long_term_threshold,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure,multibranch_party_A,multibranch_party_B,governing law
0,5_bk,20021028,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,1,...,below,NotFound,below,BBB+,Fixed Amount,10000000,NotFound,worldwide,No,English Law
1,16_bk,19910919,SOCIETE GENERALE,Bank of ABC,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,BBB-,NotFound,NotFound,Fixed Amount,10000000,NotFound,No,No,English Law
2,21_bk,19850725,SOCIETE GENERALE,Company ABC,AUD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,No,No,English Law
3,24_bk,Not Found,SOCIETE GENERALE,Societe Generale,USD,NotFound,Party B,substitution calculation agent in case of dispute,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,of,No,English Law
4,25_bk,19910919,SOCIETE GENERALE,Bank of ABC,AUD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,NotFound,below,BBB-,Fixed Amount,10000000,NotFound,No,No,English Law
5,27_bk,20040929,SOCIETE GENERALE,Societe Generale,EUR,NotFound,Party A,NotFound,yes,NotFound,...,below,NotFound,below,BBB+,Fixed Amount,10000000,NotFound,of,worldwide,English Law
6,28_bk,20030711,SOCIETE GENERALE,Company ABC,EUR,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,NotFound,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,of,athens;Greece,English Law
7,32_bk,20051201,SOCIETE GENERALE,Company ABC,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,NotFound,NotFound,below,NotFound,Fixed Amount,100000000,NotFound,paris;London;Tokyo;Sydney,No,Laws of the State of New York
8,34_bk,Not Found,SOCIETE GENERALE,Societe Generale,USD,NotFound,Party A,third party in case of default,no,NotFound,...,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,NotFound,NotFound,English Law
9,37_bk,20010924,SOCIETE GENERALE,Bank of ABC,USD,NotFound,Party A,NotFound,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,NotFound,NotFound,English Law


In [64]:
df_final_test2['date']=df_final_test2['date'].str.replace("Not Found","NotFound")

In [65]:
df_final_test2.to_csv('../final_submission_pvt.csv',index=False)

In [68]:
df_final_test2

,File_Name,date,party_A,party B,TC_currency,TC_Bespoke,calulation_agent,CA_fallback_default_dispute,CA_dispute_resolution,RD_method_party_A,...,RD_party_A_SnP_long_term_trigger_method,RD_party_A_SnP_long_term_threshold,RD_party_A_Fitch_long_term_trigger_method,RD_party_A_Fitch_long_term_threshold,Cross_default_threshold_type,Cross_default_percentage/amount,Cross_default_measure,multibranch_party_A,multibranch_party_B,governing law
0,5_bk,20021028,SOCIETE GENERALE,Societe Generale,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,1,...,below,NotFound,below,BBB+,Fixed Amount,10000000,NotFound,worldwide,No,English Law
1,16_bk,19910919,SOCIETE GENERALE,Bank of ABC,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,BBB-,NotFound,NotFound,Fixed Amount,10000000,NotFound,No,No,English Law
2,21_bk,19850725,SOCIETE GENERALE,Company ABC,AUD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,No,No,English Law
3,24_bk,NotFound,SOCIETE GENERALE,Societe Generale,USD,NotFound,Party B,substitution calculation agent in case of dispute,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,of,No,English Law
4,25_bk,19910919,SOCIETE GENERALE,Bank of ABC,AUD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,below,NotFound,below,BBB-,Fixed Amount,10000000,NotFound,No,No,English Law
5,27_bk,20040929,SOCIETE GENERALE,Societe Generale,EUR,NotFound,Party A,NotFound,yes,NotFound,...,below,NotFound,below,BBB+,Fixed Amount,10000000,NotFound,of,worldwide,English Law
6,28_bk,20030711,SOCIETE GENERALE,Company ABC,EUR,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,NotFound,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,of,athens;Greece,English Law
7,32_bk,20051201,SOCIETE GENERALE,Company ABC,USD,To be selected by the Non-defaulting party or ...,Party A,NotFound,no,NotFound,...,NotFound,NotFound,below,NotFound,Fixed Amount,100000000,NotFound,paris;London;Tokyo;Sydney,No,Laws of the State of New York
8,34_bk,NotFound,SOCIETE GENERALE,Societe Generale,USD,NotFound,Party A,third party in case of default,no,NotFound,...,below,NotFound,NotFound,NotFound,Fixed Amount,10000000,NotFound,NotFound,NotFound,English Law
9,37_bk,20010924,SOCIETE GENERALE,Bank of ABC,USD,NotFound,Party A,NotFound,no,NotFound,...,below,A,NotFound,NotFound,Fixed Amount,10000000,NotFound,NotFound,NotFound,English Law
